adapted from <https://developmentalsystems.org/sensorimotor-lenia/>


In [ ]:
import time

from addict import Dict
import numpy as np
import torch

from pylib.difflenia._LeniaInitializationSpace import LeniaInitializationSpace
from pylib.difflenia._Lenia_C import Lenia_C
from pylib.difflenia._VideoWriter import VideoWriter

torch.set_default_tensor_type("torch.FloatTensor")


In [ ]:
# 3 examples of discovered creatures (1 channel and 10 rules)

!wget  'https://osf.io/tqxhu/download' -O 'crea1.pickle'
!wget  'https://osf.io/tqfwk/download' -O 'crea2.pickle'
!wget  'https://osf.io/ba536/download' -O 'crea3.pickle'


In [ ]:
# Ensure the device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import cv2
import matplotlib.cm as cm


def main(
    SX,
    SY,
    mode,
    borders,
    list_kernels=range(10),
    creaFile="crea1.pickle",
    modeb="none",
    zoom=1,
):
    lenia_config = Lenia_C.default_config()
    lenia_config.SX = SX
    lenia_config.SY = SY
    lenia_config.final_step = 1000
    lenia_config.version = "pytorch_fft"
    lenia_config.nb_kernels = len(list_kernels)
    initialization_space_config = Dict()
    initialization_space = LeniaInitializationSpace(
        config=initialization_space_config
    )
    system = Lenia_C(
        initialization_space=initialization_space,
        config=lenia_config,
        device=device,
    )
    a = torch.load(creaFile, map_location=torch.device(device))

    system.init_loca = []
    for x, y in (1, 1), (100, 100), (1, 100):
        for i in range(x, x + 40):
            for j in range(y, y + 40):
                system.init_loca.append((i, j))

    # b=torch.load("run_0000179_data.pickle")
    policy_parameters = Dict.fromkeys(["initialization", "update_rule"])
    policy_parameters["initialization"] = a["policy_parameters"][
        "initialization"
    ]
    policy_parameters["update_rule"] = a["policy_parameters"]["update_rule"]

    # random_kernels=torch.randperm(10)[:9]

    policy_parameters["update_rule"]["R"] = (
        policy_parameters["update_rule"]["R"] + 15
    ) * zoom - 15
    init_s = policy_parameters["initialization"].init.cpu().numpy() * 1.0

    width = int(init_s.shape[1] * zoom)
    height = int(init_s.shape[0] * zoom)
    dim = (width, height)
    # resize image
    resized = cv2.resize(init_s, dim)
    init_f = torch.tensor(resized).to(device)

    for k in policy_parameters["update_rule"].keys():

        if k != "R" and k != "T":

            policy_parameters["update_rule"][k] = policy_parameters[
                "update_rule"
            ][k][list_kernels]
        policy_parameters["update_rule"][k] = policy_parameters["update_rule"][
            k
        ].to(device)

    system.reset(
        initialization_parameters=policy_parameters["initialization"],
        update_rule_parameters=policy_parameters["update_rule"],
    )

    for once in (None,):

        if mode == "random":
            nb_obstacle = 20
            system.random_obstacle(nb_obstacle)

        if borders:
            system.init_wall[:, :4] = 1
            system.init_wall[:, -4:] = 1
            system.init_wall[-4:, :] = 1
            system.init_wall[:4, :] = 1
        print("Lenia running")
        time_b = time.time()
        with torch.no_grad():
            system.generate_init_state()
            system.state[0, :, :, 0] = 0
            #   print(system.init_loca)
            for loca in system.init_loca:
                i, j = loca
                if True:
                    system.state[
                        0,
                        loca[0] : loca[0] + init_f.shape[0],
                        loca[1] : loca[1] + init_f.shape[1],
                        0,
                    ] = init_f
            observations = system.run()

        print("Creating video")

        time_lenia = time.time() - time_b

        cm.get_cmap("jet")
        with VideoWriter("out.gif", 30.0) as vid:
            for timestep in range(observations["states"].shape[0]):

                # rgb_im = im_from_array_with_colormap(a["states"][timestep,:,:,0].detach().cpu().numpy(), colormap)
                # rgb_im = np.array(rgb_im.convert("RGB"))
                # rgb_arr = np.array(rgb_im.convert("RGB"))
                # print(a["states"][timestep,:,:,0].detach().cpu().unsqueeze(-1).numpy().repeat(2,2).shape)

                rgb_im = np.concatenate(
                    [
                        observations["states"][timestep, :, :, 0]
                        .detach()
                        .cpu()
                        .unsqueeze(-1)
                        .numpy()
                        .repeat(2, 2),
                        observations["states"][timestep, :, :, 1]
                        .detach()
                        .cpu()
                        .unsqueeze(-1)
                        .numpy(),
                    ],
                    axis=2,
                )
                # rgb_im=cmap(observations["states"][timestep,:,:,0].detach().cpu().numpy())[:,:,:3]
                # rgb_im=np.clip(rgb_im-observations["states"][timestep,:,:,1].detach().cpu().unsqueeze(-1).numpy(),0,1)
                vid.add(rgb_im)
            # clear_output(wait=False)
            #   print(policy_parameters['update_rule'])
            vid.show()

        print("computation of lenia took " + str(time_lenia))


In [ ]:
for i in range(2):
    # @title Main
    SY = 256  # @param {type:"integer"}
    SX = 256  # @param {type:"integer"}
    # timesteps =  200#@param {type:"integer"}
    mode = "random"  # @param ["draw", "random"]
    modeb = "none"  # @param ["growth", "sum", "none", "both"]
    borders = True  # @param {type:"boolean"}
    creature = "crea1"  # @param ["crea1","crea2","crea3"]
    main(
        SX,
        SY,
        mode,
        borders,
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        creaFile=creature + ".pickle",
        modeb=modeb,
        zoom=1,
    )


In [ ]:
for i in range(2):
    # @title Main
    SY = 256  # @param {type:"integer"}
    SX = 256  # @param {type:"integer"}
    # timesteps =  200#@param {type:"integer"}
    mode = "random"  # @param ["draw", "random"]
    modeb = "none"  # @param ["growth", "sum", "none", "both"]
    borders = True  # @param {type:"boolean"}
    creature = "crea2"  # @param ["crea1","crea2","crea3"]
    main(
        SX,
        SY,
        mode,
        borders,
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        creaFile=creature + ".pickle",
        modeb=modeb,
        zoom=1,
    )


In [ ]:
for i in range(4):
    # @title Main
    SY = 256  # @param {type:"integer"}
    SX = 256  # @param {type:"integer"}
    # timesteps =  200#@param {type:"integer"}
    mode = "random"  # @param ["draw", "random"]
    modeb = "none"  # @param ["growth", "sum", "none", "both"]
    borders = False  # @param {type:"boolean"}
    creature = "crea3"  # @param ["crea1","crea2","crea3"]
    main(
        SX,
        SY,
        mode,
        borders,
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        creaFile=creature + ".pickle",
        modeb=modeb,
        zoom=1,
    )
